# RoboGraph
This is the demo for the submission of paper
>  __Certified Robustness of Graph Convolution Networks for Graph Classification under Topological Attacks__

Before running the demo, please make sure all the required packages are installed.

A detailed instruction is provided in [README.md](./README.md).

In [12]:
import torch
import numpy as np
import os.path as osp
import random
import tempfile
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_geometric.data.makedirs import makedirs
from robograph.model.gnn import GC_NET, train, eval
from tqdm.notebook import tqdm
from robograph.utils import process_data, cal_logits

from robograph.attack.admm import admm_solver_doubleL
from robograph.attack.cvx_env_solver import cvx_env_solver
from robograph.attack.dual import dual_solver_doubleL
from robograph.attack.greedy_attack import Greedy_Attack
from robograph.attack.utils import calculate_Fc
from robograph.attack.convex_relaxation import ConvexRelaxation

## Graph classification with _ReLU_ activation function

In [24]:
torch.manual_seed(0)
np.random.seed(0)

# 2. Load original TUDataset
ds_name = 'ENZYMES'
path = osp.join(tempfile.gettempdir(), 'data', ds_name)
dataset = TUDataset(path, name=ds_name, use_node_attr=True)

# 3. Store dataset-wide num_features / num_classes
n_features = dataset.num_features
n_classes = dataset.num_classes

# 4. Filter out large graphs (>31 nodes)
max_nodes = 31
filtered_data_list = []
for data in dataset:
    if data.num_nodes <= max_nodes:
        filtered_data_list.append(data)

# 5. Shuffle & split the filtered list
random.shuffle(filtered_data_list)
train_size = len(filtered_data_list) // 10 * 3  # 30%
val_size   = len(filtered_data_list) // 10 * 2  # 20%
train_dataset = filtered_data_list[:train_size]
val_dataset   = filtered_data_list[train_size: train_size + val_size]
test_dataset  = filtered_data_list[train_size + val_size:]

# 6. Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=20)
val_loader   = DataLoader(val_dataset,   batch_size=20)
test_loader  = DataLoader(test_dataset,  batch_size=20)

# 7. Pick device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 8. Create the model using the stored n_features / n_classes
model = GC_NET(
    hidden=64,
    n_features=n_features,
    n_classes=n_classes,
    act='relu',
    pool='avg',
    dropout=0.
).to(device)

# torch.manual_seed(0)
# np.random.seed(0)

# # prepare dataset
# ds_name = 'ENZYMES'
# path = osp.join(tempfile.gettempdir(), 'data', ds_name)
# save_path = osp.join(tempfile.gettempdir(), 'data', ds_name, 'saved')
# if not osp.isdir(save_path):
#     makedirs(save_path)
# dataset = TUDataset(path, name=ds_name, use_node_attr=True)
# dataset = dataset.shuffle()
# train_size = len(dataset) // 10 * 3
# val_size = len(dataset) // 10 * 2
# train_dataset = dataset[:train_size]
# val_dataset = dataset[train_size: train_size + val_size]
# test_dataset = dataset[train_size + val_size:]

# # prepare dataloader
# train_loader = DataLoader(train_dataset, batch_size=20)
# val_loader = DataLoader(val_dataset, batch_size=20)
# test_loader = DataLoader(test_dataset, batch_size=20)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # create model
# model = GC_NET(hidden=64,
#               n_features=dataset.num_features,
#               n_classes=dataset.num_classes,
#               act='relu',
#               pool='avg',
#               dropout=0.).to(device)

## Training a vanilla model

In [25]:
best=0
for epoch in tqdm(range(200)):
    loss_all = train(model, train_loader)
    train_acc = eval(model, train_loader)
    val_acc = eval(model, val_loader)
    if val_acc >= best:
        best = val_acc
        torch.save(model.state_dict(), osp.join(save_path, "result.pk"))
    
    tqdm.write("epoch {:03d} ".format(epoch+1) + 
              "train_loss {:.4f} ".format(loss_all) +
              "train_acc {:.4f} ".format(train_acc) +
              "val_acc {:.4f} ".format(val_acc))
test_acc = eval(model, test_loader, testing=True, save_path=save_path)
print("test_acc {:.4f}".format(test_acc))

  0%|          | 0/200 [00:00<?, ?it/s]

epoch 001 train_loss 3.8261 train_acc 0.2167 val_acc 0.2083 
epoch 002 train_loss 2.8495 train_acc 0.2389 val_acc 0.2000 
epoch 003 train_loss 2.0438 train_acc 0.2500 val_acc 0.2167 
epoch 004 train_loss 1.8967 train_acc 0.1889 val_acc 0.2167 
epoch 005 train_loss 1.8471 train_acc 0.2500 val_acc 0.2250 
epoch 006 train_loss 1.8686 train_acc 0.1889 val_acc 0.1917 
epoch 007 train_loss 1.8205 train_acc 0.2556 val_acc 0.3000 
epoch 008 train_loss 1.8513 train_acc 0.1833 val_acc 0.2417 
epoch 009 train_loss 1.8041 train_acc 0.2667 val_acc 0.3000 
epoch 010 train_loss 1.8387 train_acc 0.2167 val_acc 0.2333 
epoch 011 train_loss 1.7913 train_acc 0.2722 val_acc 0.2833 
epoch 012 train_loss 1.8251 train_acc 0.2444 val_acc 0.2417 
epoch 013 train_loss 1.7802 train_acc 0.2889 val_acc 0.2833 
epoch 014 train_loss 1.8148 train_acc 0.2500 val_acc 0.2250 
epoch 015 train_loss 1.7713 train_acc 0.2833 val_acc 0.2917 
epoch 016 train_loss 1.8055 train_acc 0.2722 val_acc 0.2333 
epoch 017 train_loss 1.7

In [ ]:
epoch 200 train_loss 1.3260 train_acc 0.4943 val_acc 0.2759 
test_acc 0.3487

## Robustness certificate

In [16]:
W = model.conv.weight.detach().cpu().numpy().astype(np.float64)
U = model.lin.weight.detach().cpu().numpy().astype(np.float64)

k = dataset.num_classes

# counter of certifiably robust and vulnerable 
robust_dual = 0
robust_cvx_DL = 0
robust_cvx_SL = 0
vul_admm_g = 0
vul_greedy = 0

# counter of correct classification
correct = 0

# attacker settings
strength = 2
delta_g = 5

# setting for solvers
dual_params = dict(iter=100, verbose=0, nonsmooth_init='random')
cvx_params = dict(iter=400, lr=0.3, verbose=0, constr='1+2+3', 
                 activation='relu', algo='swapping', nonsmooth_init='subgrad')
admm_params = dict(iter=200, mu=1)

for data in tqdm(test_dataset, desc='across graphs'):
    A, X, y = process_data(data)
    deg = A.sum(1)
    n_nodes = A.shape[0]
    n_edges = np.count_nonzero(A) // 2
    
    delta_l = np.minimum(np.maximum(deg - np.max(deg) + strength, 0), n_nodes - 1).astype(int)
    # delta_g
    
    logits = cal_logits(A, X@W, U, act='relu')
    c_pred = logits.argmax()
    
    if c_pred != y:
        continue
    correct += 1
    fc_vals_orig = [0] * k
    fc_vals_dual = [0] * k
    fc_vals_cvx_DL = [0] * k
    fc_vals_cvx_SL = [0] * k
    fc_vals_admm = [0] * k
    fc_vals_admm_g = [0] * k
    fc_vals_greedy = [0] * k
    
    
    for c in tqdm(range(k), desc='across labels', leave=False):
        if c == y:
            continue
        u = U[y] - U[c]
        XW = X@W
        
        # fc_val_orig
        fc_vals_orig[c] = calculate_Fc(A, XW, u / n_nodes, activation='relu')
        
        # fc_val_dual
        cvx_relax = ConvexRelaxation(A, XW, u / n_nodes, delta_l, delta_g, 'relu', 'envelop', 'doubleL')
        dual_sol = dual_solver_doubleL(A, cvx_relax.Q, cvx_relax.p, 
                                       delta_l=delta_l, delta_g=delta_g, **dual_params)
        fc_vals_dual[c] = dual_sol['opt_f']
        
        # fc_val_cvx_DL
        cvx_env_params = dict(iter=100, lr=1, verbose=0, constr='1+2+3', nonsmooth_init='subgrad',
                                  activation='relu', algo='swapping', relu_bound='doubleL')
        cvx_env_doubleL_sol = cvx_env_solver(A, XW, u / n_nodes, delta_l, delta_g, **cvx_env_params)
        fc_vals_cvx_DL[c] = cvx_env_doubleL_sol['opt_f']
        
        # fc_val_cvx_SL
        cvx_env_params = dict(iter=100, lr=1, verbose=0, constr='1+2+3', nonsmooth_init='subgrad',
                                  activation='relu', algo='swapping', relu_bound='singleL')
        cvx_env_singleL_sol = cvx_env_solver(A, XW, u / n_nodes, delta_l, delta_g, **cvx_env_params)
        fc_vals_cvx_SL[c] = cvx_env_singleL_sol['opt_f']
        
        # fc_val_admm
        Q, p = cvx_relax.doubleL_ub_coefficient()
        admm_params['init_B'] = dual_sol['opt_A'].copy()
        admm_sol = admm_solver_doubleL(A, Q, p, delta_l, delta_g, **admm_params)
        fc_vals_admm[c] = admm_sol['opt_f']
        
        # fc_val_admm_g: admm + greedy
        admm_g_attack = Greedy_Attack(A, XW, u / n_nodes, delta_l=delta_l, delta_g=delta_g, activation='relu')
        if np.array_equal(admm_sol['opt_A'], admm_sol['opt_A'].T):
            admm_A = admm_sol['opt_A']
        else:
            admm_A = np.minimum(admm_sol['opt_A'], admm_sol['opt_A'].T)
        admm_g_sol = admm_g_attack.attack(admm_A)  # init from A
        fc_vals_admm_g[c] = admm_g_sol['opt_f']
        
        # fc_val_greedy
        attack = Greedy_Attack(A, XW, u / n_nodes, delta_l=delta_l, delta_g=delta_g, activation='relu')
        greedy_sol = attack.attack(A)  # init from A
        fc_vals_greedy[c] = greedy_sol['opt_f']
    
    if np.min(fc_vals_dual) >= 0:
        robust_dual += 1
    if np.min(fc_vals_cvx_DL) >= 0:
        robust_cvx_DL += 1
    if np.min(fc_vals_cvx_SL) >= 0:
        robust_cvx_SL += 1
    if np.min(fc_vals_admm_g) < 0:
        vul_admm_g += 1
    if np.min(fc_vals_greedy) < 0:
        vul_greedy += 1

across graphs:   0%|          | 0/152 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

...

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
print('dataset {}'.format(ds_name),
      'strength {:02d}'.format(strength),
      'delta_g {:02d}'.format(delta_g),
      'dual {:.3f}'.format(robust_dual / correct),
      'cvx_DL {:.3f}'.format(robust_cvx_DL / correct),
      'cvx_SL {:.3f}'.format(robust_cvx_SL / correct),
      'admm_g {:.3f}'.format(vul_admm_g / correct),
      'greedy {:.3f}'.format(vul_greedy / correct),
      )

dataset ENZYMES strength 02 delta_g 05 dual 0.962 cvx_DL 0.962 cvx_SL 0.962 admm_g 0.019 greedy 0.019


## Warm start from adversarial sample by greedy method

In [18]:
strength = 3
for idx, data in tqdm(enumerate(train_dataset), desc='adverarial examples'):
    A, X, y = process_data(data)
    deg = A.sum(1)
    n_nodes = A.shape[0]
    delta_l = np.minimum(np.maximum(deg - np.max(deg) + strength, 0), n_nodes - 1).astype(int)
    delta_g = n_nodes * np.max(delta_l)
    logits = cal_logits(A, X@W, U, act='relu')
    c_pred = logits.argmax()
    
    fc_vals_greedy = [0] * k
    fc_A_greedy = [A] * k
    for c in range(k):
        u = U[y] - U[c]
        XW = X@W
        ''' greedy attack '''
        attack = Greedy_Attack(A, XW, u / n_nodes, delta_l=delta_l, delta_g=delta_g)
        greedy_sol = attack.attack(A)  # init from A
        fc_vals_greedy[c] = greedy_sol['opt_f']
        fc_A_greedy[c] = greedy_sol['opt_A']
    pick_idx = np.argmin(fc_vals_greedy)
    train_dataset[idx].edge_index = torch.tensor(fc_A_greedy[pick_idx].nonzero())
torch.save(train_dataset, osp.join(save_path, 'adv_set.pk'))

adverarial examples: 0it [00:00, ?it/s]

## Robust ReLU model

In [19]:
model = GC_NET(hidden=64,
               n_features=dataset.num_features,
               n_classes=dataset.num_classes,
               act='relu',
               pool='avg',
               dropout=0.).to(device)
adv = torch.load(osp.join(save_path, 'adv_set.pk'))
adv_loader = DataLoader(adv + train_dataset, batch_size=20)

best = 0
for epoch in tqdm(range(200), desc='epoch'):
    loss_all =  train(model, train_loader, robust=True, adv_loader=adv_loader, lamb=0.5)
    train_acc = eval(model, train_loader, robust=True)
    val_acc = eval(model, val_loader, robust=True)
    
    if val_acc >= best:
        best = val_acc
        torch.save(model.state_dict(), osp.join(save_path, 'result_robust.pk'))
    #     tqdm.write("epoch {:03d} ".format(epoch+1) + 
    #               "train_loss {:.4f} ".format(loss_all) +
    #               "train_acc {:.4f} ".format(train_acc) +
    #               "val_acc {:.4f} ".format(val_acc))

test_acc = eval(model, test_loader, testing=True, save_path=save_path, robust=True)
print("test_acc {:.4f}".format(test_acc))

/home/kendra/miniconda3/envs/robograph2/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


epoch:   0%|          | 0/200 [00:00<?, ?it/s]

test_acc 0.3224


## Robustness certificate with robust model

In [20]:
W = model.conv.weight.detach().cpu().numpy().astype(np.float64)
U = model.lin.weight.detach().cpu().numpy().astype(np.float64)

k = dataset.num_classes

# counter of certifiably robust and vulnerable 
robust_dual = 0
robust_cvx_DL = 0
robust_cvx_SL = 0
vul_admm_g = 0
vul_greedy = 0

# counter of correct classification
correct = 0

# attacker settings
strength = 3
delta_g = 10

# setting for solvers
dual_params = dict(iter=100, verbose=0, nonsmooth_init='random')
cvx_params = dict(iter=400, lr=0.3, verbose=0, constr='1+2+3', 
                 activation='relu', algo='swapping', nonsmooth_init='subgrad')
admm_params = dict(iter=200, mu=1)

for data in tqdm(test_dataset, desc='across graphs'):
    A, X, y = process_data(data)
    deg = A.sum(1)
    n_nodes = A.shape[0]
    n_edges = np.count_nonzero(A) // 2
    
    delta_l = np.minimum(np.maximum(deg - np.max(deg) + strength, 0), n_nodes - 1).astype(int)
    # delta_g
    
    logits = cal_logits(A, X@W, U, act='relu')
    c_pred = logits.argmax()
    
    if c_pred != y:
        continue
    correct += 1
    fc_vals_orig = [0] * k
    fc_vals_dual = [0] * k
    fc_vals_cvx_DL = [0] * k
    fc_vals_cvx_SL = [0] * k
    fc_vals_admm = [0] * k
    fc_vals_admm_g = [0] * k
    fc_vals_greedy = [0] * k
    
    
    for c in tqdm(range(k), desc='across labels', leave=False):
        if c == y:
            continue
        u = U[y] - U[c]
        XW = X@W
        
        # fc_val_orig
        fc_vals_orig[c] = calculate_Fc(A, XW, u / n_nodes, activation='relu')
        
        # fc_val_dual
        cvx_relax = ConvexRelaxation(A, XW, u / n_nodes, delta_l, delta_g, 'relu', 'envelop', 'doubleL')
        dual_sol = dual_solver_doubleL(A, cvx_relax.Q, cvx_relax.p, 
                                       delta_l=delta_l, delta_g=delta_g, **dual_params)
        fc_vals_dual[c] = dual_sol['opt_f']
        
        # fc_val_cvx_DL
        cvx_env_params = dict(iter=100, lr=1, verbose=0, constr='1+2+3', nonsmooth_init='subgrad',
                                  activation='relu', algo='swapping', relu_bound='doubleL')
        cvx_env_doubleL_sol = cvx_env_solver(A, XW, u / n_nodes, delta_l, delta_g, **cvx_env_params)
        fc_vals_cvx_DL[c] = cvx_env_doubleL_sol['opt_f']
        
        # fc_val_cvx_SL
        cvx_env_params = dict(iter=100, lr=1, verbose=0, constr='1+2+3', nonsmooth_init='subgrad',
                                  activation='relu', algo='swapping', relu_bound='singleL')
        cvx_env_singleL_sol = cvx_env_solver(A, XW, u / n_nodes, delta_l, delta_g, **cvx_env_params)
        fc_vals_cvx_SL[c] = cvx_env_singleL_sol['opt_f']
        
        # fc_val_admm
        Q, p = cvx_relax.doubleL_ub_coefficient()
        admm_params['init_B'] = dual_sol['opt_A'].copy()
        admm_sol = admm_solver_doubleL(A, Q, p, delta_l, delta_g, **admm_params)
        fc_vals_admm[c] = admm_sol['opt_f']
        
        # fc_val_admm_g: admm + greedy
        admm_g_attack = Greedy_Attack(A, XW, u / n_nodes, delta_l=delta_l, delta_g=delta_g, activation='relu')
        if np.array_equal(admm_sol['opt_A'], admm_sol['opt_A'].T):
            admm_A = admm_sol['opt_A']
        else:
            admm_A = np.minimum(admm_sol['opt_A'], admm_sol['opt_A'].T)
        admm_g_sol = admm_g_attack.attack(admm_A)  # init from A
        fc_vals_admm_g[c] = admm_g_sol['opt_f']
        
        # fc_val_greedy
        attack = Greedy_Attack(A, XW, u / n_nodes, delta_l=delta_l, delta_g=delta_g, activation='relu')
        greedy_sol = attack.attack(A)  # init from A
        fc_vals_greedy[c] = greedy_sol['opt_f']
    
    if np.min(fc_vals_dual) >= 0:
        robust_dual += 1
    if np.min(fc_vals_cvx_DL) >= 0:
        robust_cvx_DL += 1
    if np.min(fc_vals_cvx_SL) >= 0:
        robust_cvx_SL += 1
    if np.min(fc_vals_admm_g) < 0:
        vul_admm_g += 1
    if np.min(fc_vals_greedy) < 0:
        vul_greedy += 1

across graphs:   0%|          | 0/152 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

...

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

across labels:   0%|          | 0/6 [00:00<?, ?it/s]

In [21]:
print('dataset {}'.format(ds_name),
      'strength {:02d}'.format(strength),
      'delta_g {:02d}'.format(delta_g),
      'dual {:.3f}'.format(robust_dual / correct),
      'cvx_DL {:.3f}'.format(robust_cvx_DL / correct),
      'cvx_SL {:.3f}'.format(robust_cvx_SL / correct),
      'admm_g {:.3f}'.format(vul_admm_g / correct),
      'greedy {:.3f}'.format(vul_greedy / correct),
      )

dataset ENZYMES strength 03 delta_g 10 dual 0.735 cvx_DL 0.735 cvx_SL 0.755 admm_g 0.163 greedy 0.163
